In [4]:
import pandas as pd
from pandasql import sqldf

# Sample data - 3 
orders = pd.DataFrame({
    'id': [1000, 2000],
    'city': ['Barcelona', 'Madrid'],
    'user_id': [1, 2],
    'Gen1': [1, 0],
    'category': ['FOOD', 'GROCERIES'],
    'order_date': ['2024-01-14 21:10:34.000000', '2024-01-18 21:10:34.000000']
})

users = pd.DataFrame({
    'id': [1, 2],
    'city': ['Barcelona', 'Madrid'],
    'registration_date': ['2024-01-05 09:40:29.000000', '2023-12-23 13:25:47.000000'],
    'first_order_date': ['2023-10-12 08:20:46.000000', '2023-11-07 08:20:46.000000']
})

# Convert string representations to datetime objects
orders['order_date'] = pd.to_datetime(orders['order_date'])
users['registration_date'] = pd.to_datetime(users['registration_date'])
users['first_order_date'] = pd.to_datetime(users['first_order_date'])

In [6]:
from pandasql import sqldf
import pandas as pd

# Define a pandasql function
pysqldf = lambda q: sqldf(q, globals())

# Assuming you have orders and users DataFrame
# Define orders and users DataFrame here

# SQL-like query
query = """
WITH CityGroupData AS (
    SELECT 
        CASE 
            WHEN o.city IN ('Barcelona') THEN 'Group1'
            WHEN o.city IN ('Madrid') THEN 'Group2'
            WHEN o.city IN ('Valencia', 'Murcia') THEN 'Group3'
            WHEN o.city NOT IN ('Barcelona', 'Madrid', 'Valencia', 'Murcia') AND o.Gen1 = 0 THEN 'Group4'
            ELSE NULL
        END AS CityGroup,
        o.id AS order_id,
        u.id AS user_id,
        o.order_date,
        u.registration_date,
        u.first_order_date,
        o.category
    FROM 
        orders o
    JOIN 
        users u ON o.user_id = u.id
),
LastWeekOrders AS (
    SELECT 
        CityGroup,
        COUNT(DISTINCT CASE WHEN order_date >= date('now', '-7 days') AND order_date < date('now') THEN order_id END) AS LastWeekNumOrders,
        COUNT(DISTINCT CASE WHEN order_date >= date('now', '-14 days') AND order_date < date('now', '-7 days') THEN order_id END) AS LastWeekNumOrdersPrev
    FROM 
        CityGroupData
    GROUP BY 
        CityGroup
),
LastWeekRegistrations AS (
    SELECT 
        CityGroup,
        COUNT(DISTINCT CASE WHEN registration_date >= date('now', '-7 days') AND registration_date < date('now') THEN user_id END) AS LastWeekNumRegistrations
    FROM 
        CityGroupData
    GROUP BY 
        CityGroup
),
LastMonthOrders AS (
    SELECT 
        CityGroup,
        SUM(CASE WHEN order_date >= date('now', '-1 month') AND order_date < date('now') AND category = 'FOOD' THEN 1 ELSE 0 END) / 
        COUNT(CASE WHEN order_date >= date('now', '-1 month') AND order_date < date('now') THEN 1 END) AS AvgNumFoodOrdersLastMonth,
        COUNT(DISTINCT CASE WHEN first_order_date < date('now', '-1 month') AND order_date < date('now') AND order_date >= date('now', '-1 month') THEN user_id END) AS LastMonthNumOldActiveUsers
    FROM 
        CityGroupData
    GROUP BY 
        CityGroup
)
SELECT 
    LWO.CityGroup,
    LWO.LastWeekNumOrders,
    LWO.LastWeekNumOrders - LWO.LastWeekNumOrdersPrev AS WoWNumOrders,
    LWR.LastWeekNumRegistrations,
    LMO.AvgNumFoodOrdersLastMonth,
    LMO.LastMonthNumOldActiveUsers
FROM 
    LastWeekOrders LWO
LEFT JOIN 
    LastWeekRegistrations LWR ON LWO.CityGroup = LWR.CityGroup
LEFT JOIN 
    LastMonthOrders LMO ON LWO.CityGroup = LMO.CityGroup;
"""

# Run the query using pandasql
result_df = pysqldf(query)

# Display the result DataFrame
result_df


,CityGroup,LastWeekNumOrders,WoWNumOrders,LastWeekNumRegistrations,AvgNumFoodOrdersLastMonth,LastMonthNumOldActiveUsers
0,Group1,0,0,0,NaN,0
1,Group2,0,0,0,0.0,1


In [ ]:
# Define a pandasql function
pysqldf = lambda q: sqldf(q, globals())

# SQL-like query
query = """
WITH LastWeekOrders AS (
    SELECT 
        CASE 
            WHEN o.city IN ('Barcelona') THEN 'Group1'
            WHEN o.city IN ('Madrid') THEN 'Group2'
            WHEN o.city IN ('Valencia', 'Murcia') THEN 'Group3'
            WHEN o.city NOT IN ('Barcelona', 'Madrid', 'Valencia', 'Murcia') AND o.Gen1 = 0 THEN 'Group4'
            ELSE NULL
        END AS CityGroup,
        COUNT(DISTINCT CASE WHEN o.order_date < DATE_TRUNC('WEEK', CURRENT_DATE) AND 
        o.order_date >= DATE_TRUNC('WEEK', CURRENT_DATE - INTERVAL '1 WEEK') THEN o.id END) AS LastWeekNumOrders,
        COUNT(DISTINCT CASE WHEN o.order_date < DATE_TRUNC('WEEK', CURRENT_DATE - INTERVAL '1 WEEK') AND 
        o.order_date >= DATE_TRUNC('WEEK', CURRENT_DATE - INTERVAL '2 WEEK') THEN o.id END) AS LastWeekNumOrdersPrev
    FROM 
        orders o
    GROUP BY 
        CityGroup
),
LastWeekRegistrations AS (
    SELECT 
        CASE 
            WHEN o.city IN ('Barcelona') THEN 'Group1'
            WHEN o.city IN ('Madrid') THEN 'Group2'
            WHEN o.city IN ('Valencia', 'Murcia') THEN 'Group3'
            WHEN o.city NOT IN ('Barcelona', 'Madrid', 'Valencia', 'Murcia') AND o.Gen1 = 0 THEN 'Group4'
            ELSE NULL
        END AS CityGroup,
        COUNT(DISTINCT CASE WHEN u.registration_date < DATE_TRUNC('WEEK', CURRENT_DATE) AND 
        u.registration_date >= DATE_TRUNC('WEEK', CURRENT_DATE - INTERVAL '1 WEEK') THEN u.id END) AS LastWeekNumRegistrations
    FROM 
        orders o
    LEFT JOIN 
        users u ON o.user_id = u.id
    GROUP BY 
        CityGroup
),
LastMonthOrders AS (
    SELECT 
        CASE 
            WHEN o.city IN ('Barcelona') THEN 'Group1'
            WHEN o.city IN ('Madrid') THEN 'Group2'
            WHEN o.city IN ('Valencia', 'Murcia') THEN 'Group3'
            WHEN o.city NOT IN ('Barcelona', 'Madrid', 'Valencia', 'Murcia') AND o.Gen1 = 0 THEN 'Group4'
            ELSE NULL
        END AS CityGroup,
        SUM(CASE WHEN o.order_date < DATE_TRUNC('MONTH', CURRENT_DATE) AND 
        o.order_date >= DATE_TRUNC('MONTH', CURRENT_DATE - INTERVAL '1 MONTH') AND 
        o.category = 'FOOD' THEN 1 ELSE 0 END) / 
        COUNT(CASE WHEN o.order_date < DATE_TRUNC('MONTH', CURRENT_DATE) AND 
        o.order_date >= DATE_TRUNC('MONTH', CURRENT_DATE - INTERVAL '1 MONTH') THEN 1 ELSE 0 END) AS AvgNumFoodOrdersLastMonth,
        COUNT(DISTINCT CASE WHEN u.first_order_date < DATE_TRUNC('MONTH', CURRENT_DATE - INTERVAL '1 MONTH') AND 
        o.order_date < DATE_TRUNC('MONTH', CURRENT_DATE) AND 
        o.order_date >= DATE_TRUNC('MONTH', CURRENT_DATE - INTERVAL '1 MONTH') THEN u.id END) AS LastMonthNumOldActiveUsers
    FROM 
        orders o
    LEFT JOIN 
        users u ON o.user_id = u.id
    GROUP BY 
        CityGroup
)
SELECT 
    LWO.CityGroup,
    LWO.LastWeekNumOrders,
    LWO.LastWeekNumOrders - LWO.LastWeekNumOrdersPrev AS WoWNumOrders,
    LWR.LastWeekNumRegistrations,
    LMO.AvgNumFoodOrdersLastMonth,
    LMO.LastMonthNumOldActiveUsers
FROM 
    LastWeekOrders LWO
LEFT JOIN 
    LastWeekRegistrations LWR ON LWO.CityGroup = LWR.CityGroup
LEFT JOIN 
    LastMonthOrders LMO ON LWO.CityGroup = LMO.CityGroup;
"""

# Execute the query
result = pysqldf(query)
print(result)